In this script we process the raw data coming from the preprocessing to standarize the data into a working Anndata file for the rest of the analysis.


In [1]:
import scanpy as scp
import scipy.sparse as sprs
import pandas as pd
import warnings
import os
import numpy as np

warnings.filterwarnings("ignore")

from parameters import *

# Gastruloid

In this step, we create the dataset for the AnnData format. 

We incorporate to the data the information coming from:

1. The raw count matrix.
2. The information coming from the flow cytometer sorting procedure.

In [2]:
genes = pd.read_csv("Data/Genes.csv")
cells = pd.read_csv("Data/Cells.csv")
m = sprs.load_npz("Data/count_matrix_total.npz")

#Adding plate information
#cells["Plate"] = 

statistics = pd.DataFrame()
statistics.loc[0,["Plates","Wells per plate","Total Wells"]] = [9,96,9*96]

## Merging lanes <a name="Merging"></a>

In the preprocessing step, we aligned the files taking the different lanes as separated measures, however, these measures corrrespond to the cells. In this step, we unified the measures coming from different lanes measures coming from a unique well. 

In [3]:
dropIndex = [] # For dropping the index that are merged
statistics.loc[0,"singleLane"] = 0
for j,i in zip(cells[cells["Lane"]==1].index,cells[cells["Lane"]==1].values):
    
    # making the conditions to univocaly finding the cell with the other corresponding lane
    name = i[0]
    time = i[2]
    condition = i[3]
    
    cond1 = cells["Cell"] == name.replace("_1_","_2_")
    cond2 = cells["Time"] == time
    cond3 = cells["Lane"] == 2

    cell = cells[cond1.values*cond2.values*cond3.values]

    #Add the
    if cell.shape[0] == 1:
        m[j,:] += m[cell.index,:]
        dropIndex.append(cell.index[0])
    else:
        print(name.replace("_1_","_2_"), " at ", time,"_",condition, " doesn't exist. We only have one lane measure of this well.")
        statistics.loc[0,"singleLane"] += 1

#Drop merged cells
cells = cells.drop(dropIndex)

#Print cells that only have measures in second lane
for j,i in zip(cells[cells["Lane"]==2].index,cells[cells["Lane"]==2].values):
    name = i[0]
    time = i[1]
    condition = i[2]
    print(name.replace("_2_","_1_"), " at ", time,"_",condition, " doesn't exist. We only have one lane measure of this well.")
    statistics.loc[0,"singleLane"] += 1
#Change name of cells that are merged
cells["Cell"] = [i.replace("_1_","_").replace("_2_","_") for i in cells["Cell"].values]

#Retain only rows that correspond to merged cells
m = m[cells.index]

#Reset index of the cells
cells.reset_index(drop=True,inplace=True)

#Drop Lane information
cells.drop(columns="Lane",inplace=True)


## Standarize the tags <a name="Standarize"></a>

We can check that in the raw data some tags have the same meaning but different nomenclature because of how they where given to the sequencing facility. We standarize the nomenclature.

1. kitScal and kitscal in Condition
2. empty and EMPTY in BiologicalSample

In [4]:
for i in cells.columns[1:]:
    print(i,": ", cells[i].unique())

Plate :  ['P1' 'P2']
Time :  ['216hr']
Condition :  ['MNX1' 'EV']
BiologicalSample :  ['1' '73' '81' '89' '2' '10' '18' '26' '34' '42' '50' '9' '58' '66' '74'
 '82' '90' '3' '11' '19' '27' '35' '17' '43' '51' '59' '67' '75' '83' '91'
 '4' '12' '20' '25' '28' '36' '44' '52' '60' '68' '76' '84' '92' '5' '33'
 '13' '21' '29' '37' '45' '53' '61' '69' '77' '85' '41' '93' '6' '14' '22'
 '30' '38' '46' '54' '62' '70' '49' '78' '86' '94' '7' '15' '23' '31' '39'
 '47' '55' '57' '63' '71' '79' '87' '95' '8' '16' '24' '32' '40' '65' '48'
 '56' '64' '72' '80' '88' 'empty']


In [5]:
cells.loc[cells["Condition"]=="kitscal","Condition"] = "kitScal"
cells.loc[cells["BiologicalSample"]=="EMPTY","BiologicalSample"] = "empty"

Additionally, we have a few cells marked with the tag black. Those correspond to measures did no have a valid measurement in the flow cytommetry system. This correspond to confirmed doublets by the flow cytometry sorting process as can be seen in the next section.

In [6]:
cells[cells["BiologicalSample"]=="black"]

,Cell,Plate,Time,Condition,BiologicalSample


In [7]:
cells[cells["Condition"]=="MNX1"]

,Cell,Plate,Time,Condition,BiologicalSample
0,HJCJ3DRXY_IDT-DUI-NXT-1,P1,216hr,MNX1,1
1,HJCJ3DRXY_IDT-DUI-NXT-10,P1,216hr,MNX1,73
2,HJCJ3DRXY_IDT-DUI-NXT-11,P1,216hr,MNX1,81
3,HJCJ3DRXY_IDT-DUI-NXT-12,P1,216hr,MNX1,89
4,HJCJ3DRXY_IDT-DUI-NXT-13,P1,216hr,MNX1,2
...,...,...,...,...,...
91,HJCJ3DRXY_IDT-DUI-NXT-92,P1,216hr,MNX1,64
92,HJCJ3DRXY_IDT-DUI-NXT-93,P1,216hr,MNX1,72
93,HJCJ3DRXY_IDT-DUI-NXT-94,P1,216hr,MNX1,80
94,HJCJ3DRXY_IDT-DUI-NXT-95,P1,216hr,MNX1,88


## Adding Flow cytommetry measures to data <a name="Add_flow"></a>

For this experimental setup, the data has been sorted to enrich for rare cells. We add the measures of the cells to the available information of the system.

We check that there are wells that have flow information but there are not corresponding RNA sequenced measures.

Additionally we confirm that some wells were measured more than once indicating that the well contains doublets, this where marked already by the tag **black** in biological condition.

In [8]:
# List the sorting files by plates
plates = os.listdir("Data/SMARTseq")
#Add identifier flow cytometry file
cells["Flow Cytommetry File"] = np.NaN

#Counters
statistics.loc[0,"doublet"] = 0
statistics.loc[0,"noRNA"] = 0
for plate in plates:

    #Load table mapping well indexes to cell indexes
    table = pd.read_excel("Data/Mapping.xlsx",sheet_name=plate).set_index("X1").unstack()

    #Make mapping list of cells to plates
    mapping = {i[1]+str(i[0]):str(j).replace("_1_","_") for i,j in zip(table.index, table.values)}

    #List and go over the sorting files splited by plate
    for file_num,file_name in enumerate(os.listdir("Data/SMARTseq/"+plate)):

        #Load table of measures for the plate and
        data = pd.read_csv("Data/SMARTseq/"+plate+"/"+file_name,skiprows=9,decimal=",")
        data.set_index("Well",inplace=True)
        
        #Add information to the cell
        parameters = [i for i in data.columns if "All" in i and "Median" in i and "Time" not in i]
        parameters_clean = [i.split("Events ")[1].split(" Median")[0] for i in parameters]
        
        for i in data.index:
                if cells[cells["Cell"] == mapping[i]].shape[0] > 0:
                    if type(cells.loc[cells["Cell"] == mapping[i],"Flow Cytommetry File"].values[0]) == str:
                        print(cells.loc[cells["Cell"] == mapping[i]].index[0],cells.loc[cells["Cell"] == mapping[i],"Cell"].values[0], " is measured twice. Doublet.")
                        statistics.loc[0,"doublet"] += 1
                    else:
                        cells.loc[cells["Cell"] == mapping[i],"Flow Cytommetry File"] = plate+"_"+str(file_num)
                        for k,j in enumerate(parameters):
                            cells.loc[cells["Cell"] == mapping[i],parameters_clean[k]] = data.loc[i,j]
                else:
                    #print(plate," ",i, " does not have scRNA data. Reading error.")
                    statistics.loc[0,"noRNA"] += 1


Finally, we check for wells that have transcription bu do not have any flow cytometry measure. Consistently, most of them correspond to wells called as empty, except for two samples that should be filled. These wells are wells that did not have any cell deposted in them.

In [9]:
statistics.loc[0,"empty"] = cells[cells["Flow Cytommetry File"].isna()*cells["BiologicalSample"]=="empty"].shape[0]
statistics.loc[0,"emptyWrong"] = cells[cells["Flow Cytommetry File"].isna()*np.array(cells["BiologicalSample"]!="empty")].shape[0]

cells[cells["Flow Cytommetry File"].isna()]

,Cell,Plate,Time,Condition,BiologicalSample,Flow Cytommetry File,FSC-A,FSC-H,FSC-W,SSC-A,SSC-H,SSC-W,PE Dazzle 594 CD41 610/20 Y/G-A,APC CD45 660/20 Red-A,APC Cy7 cKIT 780/60 Red-A,450/40 Violet-A,LssmOrange 610/20 Violet-A
2,HJCJ3DRXY_IDT-DUI-NXT-11,P1,216hr,MNX1,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,HJCJ3DRXY_IDT-DUI-NXT-96,P1,216hr,MNX1,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,HTW5GDRXY_IDT-DUI-NXT-192,P2,216hr,EV,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Statistics, create Annotated data file and save <a name="Statistics"></a>

We will save the data of the files with non-zero RNA sample measures but we will keep those measures that contain doublets and empty well for now so they appear in the QC check.

In [10]:
percentage = 0
percentage += statistics.loc[0,"singleLane"]
percentage += statistics.loc[0,"doublet"]
percentage += statistics.loc[0,"noRNA"]
percentage += statistics.loc[0,"noRNA"]
percentage += statistics.loc[0,"noRNA"]
statistics.loc[0,"wells with bad samples"] = percentage
statistics.loc[0,"wells with good samples"] = statistics.loc[0,"Total Wells"]-percentage
statistics.loc[0,"% wells with bad samples"] = statistics.loc[0,"wells with bad samples"]*100/statistics.loc[0,"Total Wells"]
statistics.loc[0,"% wells with good samples"] = statistics.loc[0,"wells with good samples"]*100/statistics.loc[0,"Total Wells"]

In [11]:
statistics

,Plates,Wells per plate,Total Wells,singleLane,doublet,noRNA,empty,emptyWrong,wells with bad samples,wells with good samples,% wells with bad samples,% wells with good samples
0,9.0,96.0,864.0,0.0,0.0,1.0,2.0,1.0,3.0,861.0,0.347222,99.652778


In [12]:
#Create the Annotated data
adata = scp.AnnData(m,obs=cells,var=genes.set_index("gene_id"))

In [13]:
adata.write("Data/Raw.h5ad")

... storing 'Plate' as categorical
... storing 'Time' as categorical
... storing 'Condition' as categorical
... storing 'BiologicalSample' as categorical
... storing 'Flow Cytommetry File' as categorical
... storing 'gene_name' as categorical
